In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [113]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [110]:
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# create doc2vec vector columns
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


ModuleNotFoundError: No module named 'wordcloud'

In [73]:
df = pd.read_excel('Downloads/Sample Data.xlsx',header=0)

In [74]:
df.head()

,Survey Year,ID,A,B,C,D,E,What is one thing Clarivate Analytics is doing well and should continue?,What is one thing you would like to see Clarivate Analytics change or improve to better meet your needs?,What is your role as a Clarivate Analytics customer?,Clarivate Products,Country,Clarivate Product Group,Region,Clarivate Business Unit
0,2019,R_1hNeF0hJOiEdZNp,Strongly Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WoSG,Belgium,Web of Science Core Collection,EMEA,Sciences
1,2019,R_2zTIGgCXSSbJ9MJ,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,WoSG,Israel,Incites,EMEA,Sciences
2,2019,R_2ANGQ9Z6T6HAQ0h,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Providing scientists with scientific informati...,NaN,End User of Products & Services,WoSG,Russian Federation,Web of Science Core Collection,EMEA,Sciences
3,2019,R_Z2Kvp3QO5Wkdu0h,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,End User of Products & Services,WoSG,United States,EndNote,Americas,Sciences
4,2019,R_1gwkxjFBAE0YUwg,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,WoSG,Brazil,Web of Science Core Collection,Americas,Sciences


In [75]:

dictionary={'Strongly Disagree':1,'Disagree':2,'Neither Agree nor Disagree':3,'Agree':4,'Strongly Agree':5}


In [76]:
df['Feedback1']=df['A'].map(dictionary)

In [77]:
df['Feedback2']=df['B'].map(dictionary)
df['Feedback3']=df['C'].map(dictionary)
df['Feedback4']=df['D'].map(dictionary)
df['Feedback5']=df['E'].map(dictionary)

In [44]:
df['Feedback5'].unique()

array([nan,  5.,  4.,  3.,  1.,  2.])

In [78]:


df.drop(df.columns[[2,3,4,5,6]], axis = 1, inplace = True) 


In [46]:
df.describe()

,Survey Year,Feedback1,Feedback2,Feedback3,Feedback4,Feedback5
count,1131.000000,1131.000000,921.000000,917.000000,917.000000,916.000000
mean,2019.526083,4.073386,4.042345,4.145038,3.659760,3.942140
std,0.499540,0.961612,0.860579,0.822520,0.916756,0.860762
min,2019.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2019.000000,4.000000,4.000000,4.000000,3.000000,4.000000
50%,2020.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,2020.000000,5.000000,5.000000,5.000000,4.000000,4.000000
max,2020.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [58]:
df.head()

,Survey Year,ID,What is one thing Clarivate Analytics is doing well and should continue?,What is one thing you would like to see Clarivate Analytics change or improve to better meet your needs?,What is your role as a Clarivate Analytics' customer?,Clarivate Products,Country,Clarivate Product Group,Region,Clarivate Business Unit,Feedback1,Feedback2,Feedback3,Feedback4,Feedback5
0,2019,R_1hNeF0hJOiEdZNp,NaN,NaN,NaN,WoSG,Belgium,Web of Science Core Collection,EMEA,Sciences,5,NaN,NaN,NaN,NaN
1,2019,R_2zTIGgCXSSbJ9MJ,NaN,NaN,NaN,WoSG,Israel,Incites,EMEA,Sciences,5,5.0,5.0,5.0,5.0
2,2019,R_2ANGQ9Z6T6HAQ0h,Providing scientists with scientific informati...,NaN,End User of Products & Services,WoSG,Russian Federation,Web of Science Core Collection,EMEA,Sciences,5,5.0,5.0,4.0,4.0
3,2019,R_Z2Kvp3QO5Wkdu0h,NaN,NaN,End User of Products & Services,WoSG,United States,EndNote,Americas,Sciences,5,5.0,5.0,5.0,5.0
4,2019,R_1gwkxjFBAE0YUwg,NaN,NaN,NaN,WoSG,Brazil,Web of Science Core Collection,Americas,Sciences,5,5.0,5.0,5.0,5.0


In [79]:
col = ['Feedback1','Feedback2','Feedback3','Feedback4','Feedback5']
for c in col:
    df[c].fillna(df[c].dropna().median(), inplace=True)
   

In [95]:
 df.rename(columns={'What is one thing Clarivate Analytics is doing well and should continue?':'positives'}, inplace=True)
df.rename(columns={'What is one thing you would like to see Clarivate Analytics change or improve to better meet your needs?':'negatives'}, inplace=True)

In [80]:
col1 = ['What is one thing Clarivate Analytics is doing well and should continue?', 'What is one thing you would like to see Clarivate Analytics change or improve to better meet your needs?','What is your role as a Clarivate Analytics customer?']
for c in col1:
    df[c].fillna('', inplace=True)
    

In [96]:
df.sample(50)

,Survey Year,ID,positives,negatives,What is your role as a Clarivate Analytics customer?,Clarivate Products,Country,Clarivate Product Group,Region,Clarivate Business Unit,Feedback1,Feedback2,Feedback3,Feedback4,Feedback5
904,2020,R_3mgx3qwAJw3oukH,Providing feedback regarding the quality of pu...,The search for publications should include bas...,End User of Products & Services,WoSG,Brazil,Journal Citation Reports,Americas,Sciences,5,5.0,5.0,4.0,4.0
78,2019,R_1rJG0JQDRTJl8OA,,,,WoSG,Ukraine,Incites,EMEA,Sciences,4,4.0,4.0,4.0,4.0
975,2020,R_1ptB3F8ZpvYyYIQ,,,,WoSG,United States,Web of Science Core Collection,Americas,Sciences,3,3.0,3.0,3.0,3.0
625,2020,R_6rHgwOGEz44KlgJ,,,,WoSG,Brazil,Web of Science Core Collection,Americas,Sciences,4,4.0,4.0,4.0,4.0
964,2020,R_3Mu4DeUm01v2jLE,,,,Life Sciences,France,Cortellis Competitive Intelligence,EMEA,Sciences,3,4.0,4.0,4.0,4.0
198,2019,R_2V9lKhjQJ36yoi9,The software is easy to use,,End User of Products & Services,WoSG,Nigeria,EndNote,EMEA,Sciences,5,5.0,5.0,4.0,5.0
1043,2020,R_86ecojMe4ynlaQF,,,,WoSG,Turkey,Web of Science Core Collection,EMEA,Sciences,5,4.0,4.0,4.0,4.0
1058,2020,R_214EWFvb494P3eK,web of science alerts,"user interface design, storage of search histo...",,WoSG,Switzerland,Web of Science Core Collection,EMEA,Sciences,3,4.0,4.0,4.0,4.0
456,2019,R_bmiuiLUVhgZkHAd,Love Endnotes! Can never live without it becau...,Can’t think of anything to change,Both,WoSG,United States,EndNote,Americas,Sciences,4,4.0,5.0,5.0,5.0
535,2019,R_8nNtyzm1aVSSuyt,,,End User of Products & Services,WoSG,Brazil,Web of Science Core Collection,Americas,Sciences,5,4.0,5.0,3.0,5.0


In [97]:
df.isnull().sum()

Survey Year                                             0
ID                                                      0
positives                                               0
negatives                                               0
What is your role as a Clarivate Analytics customer?    0
Clarivate Products                                      0
Country                                                 0
Clarivate Product Group                                 0
Region                                                  0
Clarivate Business Unit                                 0
Feedback1                                               0
Feedback2                                               0
Feedback3                                               0
Feedback4                                               0
Feedback5                                               0
dtype: int64

In [107]:
df.dtypes

Survey Year                                              int64
ID                                                      object
positives                                               object
negatives                                               object
What is your role as a Clarivate Analytics customer?    object
Clarivate Products                                      object
Country                                                 object
Clarivate Product Group                                 object
Region                                                  object
Clarivate Business Unit                                 object
Feedback1                                                int64
Feedback2                                                int32
Feedback3                                                int32
Feedback4                                                int32
Feedback5                                                int32
dtype: object

In [115]:
scores_df = pd.DataFrame(df_row, columns=['model_name', 'fold_idx', 'F1_score', 'Test_acc', 'Train_acc'])
scores_df.sort_values(by=['model_name', 'fold_idx'], inplace=True)
scores_df.reset_index(drop=True, inplace=True)

NameError: name 'df_row' is not defined

In [106]:
def evaluate(model, train_x, val_x, train_y, val_y):
    model.fit(train_x,train_y)
    pred_y = model.predict(val_x)
    train_acc = model.score(train_x, train_y)
    test_acc = accuracy_score(val_y, pred_y)
    f1_sc = f1_score(val_y, pred_y, average='weighted')
    return train_acc, test_acc, f1_sc

In [ ]:
def evaluate(model, train_x, val_x, train_y, val_y):
    model.fit(train_x,train_y)
    pred_y = model.predict(val_x)
    train_acc = model.score(train_x, train_y)
    test_acc = accuracy_score(val_y, pred_y)
    f1_sc = f1_score(val_y, pred_y, average='weighted')
    return train_acc, test_acc, f1_sc

In [117]:
df1 = pd.read_excel('Downloads/op.xlsx',header=0)

In [121]:
score_df

,model_name,fold_idx,F1_score,Test_acc,Train_acc
0,ExtraTreesClassifier,0,0.387840,0.398774,1.000000
1,ExtraTreesClassifier,1,0.399181,0.407078,1.000000
2,ExtraTreesClassifier,2,0.400834,0.411195,1.000000
3,ExtraTreesClassifier,3,0.390981,0.400198,1.000000
4,ExtraTreesClassifier,4,0.393222,0.402967,1.000000
5,ExtraTreesClassifier,5,0.391359,0.401266,1.000000
6,LGBMClassifier,0,0.403746,0.414393,0.621726
7,LGBMClassifier,1,0.412157,0.422697,0.620025
8,LGBMClassifier,2,0.398779,0.412381,0.620372
9,LGBMClassifier,3,0.402597,0.412463,0.623195
